In [30]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.document_loaders import WebBaseLoader

llm = ChatOpenAI(
    temperature=0.1, 
    model="gpt-4o-mini-2024-07-18",
)


class WikipediaUrlSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="URL을 찾기위한 질의입니다. 예시 질의: Ransomware에 대해 조사해줘."
    )


class WikipediaUrlSearchTool(BaseTool):
    name = "WikipediaUrlSearchTool"
    description = """
    인수로 질의를 받아, 검색 결과의 링크 정보를 수집합니다.
    찾은 URL 중 가장 유용하거나, 첫번째 URL을 반환합니다.

    예시 결과: https://site.com/article
    """
    args_schema: Type[
        WikipediaUrlSearchToolArgsSchema
    ] = WikipediaUrlSearchToolArgsSchema

    def _run(self, query):
        wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wiki.run(query)


class DuckDuckGoUrlSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="URL을 찾기위한 질의입니다. 예시 질의: Ransomware에 대해 조사해줘."
    )


class DuckDuckGoUrlSearchTool(BaseTool):
    name = "DuckDuckGoUrlSearchTool"
    description = """
    인수로 질의를 받아, 검색 결과의 링크 정보를 수집합니다.
    찾은 URL 중 가장 유용하거나, 첫번째 URL을 반환합니다.

    예시 결과: https://site.com/article
    """
    args_schema: Type[
        DuckDuckGoUrlSearchToolArgsSchema
    ] = DuckDuckGoUrlSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchRun()
        return ddg.run(query)


class WebResearchToolArgsSchema(BaseModel):
    url: str = Field(
        description="Wikipedia 와 DuckDuckGo에서 찾은 URL 입니다. 예시: https://site.com/article"
    )


class WebResearchTool(BaseTool):
    name = "WebResearchTool"
    description = """
    Wikipedia 와 DuckDuckGo에서 찾은 URL에서 text를 로드해서 문서로 만듭니다.
    문서의 언어는 한글이며, 문서의 마지막에 관련 URL을 제공합니다.
    인수로 URL이 1개가 주어집니다.

    인수 예시 : https://site.com/article
    """
    args_schema: Type[
        WebResearchToolArgsSchema
    ] = WebResearchToolArgsSchema

    def _run(self, url):
        wl = WebBaseLoader(url)
        return wl.load()


class SaveFileToolArgsSchema(BaseModel):
    doc: str


class SaveFileTool(BaseTool):
    name = "SaveFileTool"
    description = """
    WebResearchTool을 통해 생성된 문서를 파일에 저장합니다.
    """
    args_schema: Type[
        SaveFileToolArgsSchema
    ] = SaveFileToolArgsSchema

    def _run(self, doc):
        with open('research_doc.txt', 'w', encoding='utf-8') as f:
            f.write(doc)
        return "저장 완료"


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaUrlSearchTool(),
        DuckDuckGoUrlSearchTool(),
        WebResearchTool(),
        SaveFileTool(),
    ],
)

prompt = "XZ backdoor에 대해 조사해줘."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoUrlSearchTool` with `{'query': 'XZ backdoor'}`


What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ... The back door part comes into play with one of the main ways xz is used - SSH. SSH is an encrypted protocol between two machines where text commands can be exchanged, allowing a user to interact with a server. It's a very common utility in the Linux world and the security of this communication is critical. That operation matches the style of the XZ Utils backdoor far more than the cruder supply chain attacks of APT41 or Lazarus, by comparison. "It could very well be someone else," says Aitel. A backdoor was planted in the XZ utils package, a popular open-source compression utility for Linux/UNIX systems, by a threat actor named JiaT75. The backdoor exploited the SSH protocol to g

{'input': 'XZ backdoor에 대해 조사해줘.',
 'output': "XZ 백도어에 대한 조사 결과는 다음과 같습니다.\n\n### XZ 백도어 개요\nXZ 백도어는 2024년 2월에 리눅스 유틸리티인 xz의 liblzma 라이브러리에 악성 백도어가 도입된 사건을 말합니다. 이 백도어는 'Jia Tan'이라는 이름을 사용하는 계정에 의해 버전 5.6.0과 5.6.1에 삽입되었습니다. 이 백도어는 특정 Ed448 개인 키를 가진 공격자에게 영향을 받은 리눅스 시스템에서 원격 코드 실행 권한을 부여합니다. 이 문제는 CVE-2024-3094라는 공통 취약점 및 노출 번호가 부여되었으며, CVSS 점수는 10.0으로, 가능한 최고 점수입니다.\n\n### 배경\n이 백도어는 Microsoft 직원이자 PostgreSQL 개발자인 Andres Freund가 Debian Sid에서 성능 저하를 조사하던 중 발견했습니다. Freund는 SSH 연결이 예상보다 높은 CPU 사용량을 발생시키고 Valgrind에서 오류를 일으키는 것을 발견했습니다. 이 발견은 Openwall 프로젝트의 오픈 소스 보안 메일링 리스트에 보고되었고, 여러 소프트웨어 공급업체의 주목을 받았습니다.\n\n### 작동 방식\n악성 코드는 XZ Utils 소프트웨어 패키지의 5.6.0 및 5.6.1 릴리스에 존재합니다. 이 익스플로잇은 특정 제3자 SSH 서버 패치가 사용되지 않는 한 비활성 상태입니다. 이 간섭은 악의적인 행위자가 sshd 인증을 우회하고 전체 시스템에 원격으로 무단 접근할 수 있게 할 수 있습니다.\n\n### 대응\n미국 연방 사이버 보안 및 인프라 보안국(CISA)은 영향을 받은 장치가 이전의 안전한 버전으로 롤백할 것을 권장하는 보안 권고를 발표했습니다. 리눅스 소프트웨어 공급업체들은 영향을 받은 패키지를 이전 버전으로 되돌렸습니다. 이 사건은 오픈 소스 소프트웨어의 신뢰성에 대한 논의를 촉발했습니다.\n\n### 관련 링크\n- [XZ Utils 백도어 - 위키